### PRÉ-DIMENSIONAMENTO DAS ESTRUTURAS DO PÓRTICO

Para o pré-dimensionamento dos pilares foi empregado o método da área de influência:

 $$A_c=\frac{{N_d}^\ast}{0,85.f_{cd}+\rho \sigma_{s,\varepsilon=0,2\%}}$$ 
 
A Área de influência dos pilares do projeto é dada por:

<img src="Ainf_pil.png" alt="some text" width=50% height=50%>

-$A_1 = 3.375 m^2$

Considerando uma taxa de armadura $\rho = 2\%$. Têm a seguinte área para os pilares da base.

In [6]:
N_D = 72.2 * 1.4
F_CD = 30 / 1.4 / 10
RHO = 2 / 100
A_C = (2.5 * N_D) / (0.85 * F_CD + RHO * 42)
print("N_D: ", "%.4f" % N_D, "kN", "\nA_C: ", "%.4f" % A_C, "cm²")

N_D:  101.0800 kN 
A_C:  94.9490 cm²


### DIMENSIONAMENTO DA ESTRUTURA SEGUNDO A NBR 6118

Características geométricas da estrutura:

- $a = 4.50$ m;
- $b = 4.50$ m;
- $H = 9.00$ m;
- $h_i = 3.00$ m;
- $\lambda = 1.50$.

Para dimensionamento da estrutura de concreto de múltiplos pavimentos é necessário prever as cargas atuantes. Para isso será empregado os valores recomendados na NBR 6120 tabelas da seção 5.4 e tabela 2.

- $G_{lj} = 1.20$ kN/m²;
- $Q_{lj} = 1.50$ kN/m²;
- $G_{alv} = 1.70$ kN/m²;

Dimensões da viga (livro Yopanan Rebello):

- $b_w = 14$ cm;
- $h = 45$ cm;

Altura da alvenaria entre pavimentos:

- $H_{alv} = 3.00$ m;

Espessura da laje (livro Yopanan Rebello):

- $h_{lj} = 12$ cm;

In [1]:
import numpy as np
import FINITO
from FINITO import MEF1D

# CARREGAMENTO VIGA - CONTRIBUIÇÃO CARGA ÁREA LAJE
def SLAB_LOAD(L, H_SLAB, G_AREA, Q_AREA, A_I):
    G_SLABDEAD = 25 * H_SLAB
    G_LINEARDEAD = ((G_SLABDEAD + G_AREA) * A_I) / L
    Q_LINEARLIVE = ((Q_AREA) * A_I) / L
    return G_LINEARDEAD, Q_LINEARLIVE

# CARREGAMENTO VIGA - CONTRIBUIÇÃO ALVENARIA DE VEDAÇÃO
def WALL_DEAD_LOAD(H_WALL, G_WALL):
    G_WALLDEAD = G_WALL * H_WALL
    return G_WALLDEAD

# CARREGAMENTO VIGA - CONTRIBUIÇÃO DO PESO PRÓPRIO
def BEAM_DEAD_LOAD(B_W, H):
    G_BEAMDEAD = 25 * B_W * H
    return G_BEAMDEAD

# FATOR DO AGREGADO
def AGGREGATE(AGGREGATE_TYPE):
    if AGGREGATE_TYPE == 0:
        ALPHA_E = 1.20
    elif AGGREGATE_TYPE == 1:
        ALPHA_E = 1.00
    elif AGGREGATE_TYPE == 2:
        ALPHA_E = 0.90
    elif AGGREGATE_TYPE == 3:
        ALPHA_E = 0.70
    return ALPHA_E

# MÓDULO DE ELASTICIDADE TANGENTE DO CONCRETO
def TANGENT_YOUNG_MODULUS(F_CK, AGGREGATE_TYPE):
    ALPHA_E = AGGREGATE(AGGREGATE_TYPE)
    if F_CK >= 20 and F_CK <= 50:
        E_CI = ALPHA_E * 5600 * np.sqrt(F_CK)
    elif F_CK > 50 and F_CK <= 90:
        E_CI = 21.5E3 * ALPHA_E * (F_CK / 10 + 1.25) ** (1 / 3)
    return E_CI

# MÓDULO DE ELASTICIDADE SECANTE DO CONCRETO
def SECANT_YOUNG_MODULUS(F_CK, E_CI):
    ALPHA_I = 0.80 + 0.20 * (F_CK / 80)
    if ALPHA_I > 1.00:
        ALPHA_I = 1.00
    E_CS = ALPHA_I * E_CI
    return E_CS

In [2]:
# TESTE
# ESTE PARÂMETRO É O SAMPLE QUE VÊM DO ALGORITMO DA PLATAFORMA RASD
X = [1.2, 1.5, 30, 3, 1, 1]

# ATRIBUIÇÃO DOS CARREGAMENTOS
G_RASD = X[0]
Q_RASD = X[1]

# PARÂMETROS GEOMÉTRICOS
L = 4.5
H_LJ = 0.12
A_1 = 4.5
H_PAR = 3
G_PAR = 1.7
B_W = 0.14
H = 0.40
A_BEAM = B_W * H
A_COLUMN = B_W * H

# CARGA LINEAR NA VIGA
# PARCELA LAJE
[G_1, Q_1] = SLAB_LOAD(L, H_LJ, G_RASD, Q_RASD, A_1)
# PARCELA PAREDE
G_2 = WALL_DEAD_LOAD(H_PAR, G_PAR)
# PARCELA PP VIGA
G_3 = BEAM_DEAD_LOAD(B_W, H)
# TOTAL G E Q
G = G_1 + G_2 + G_3
Q = Q_1
print("carga permanente linear: ", "%.4f" % G, "kN/m", "\ncarga variável linear: ", "%.4f" % Q, "kN/m")

# PARÂMETROS DE RESISTÊNCIA
F_CK = X[2]
F_Y = X[3]

# PROP. MECÂNICAS DO CONCRETO
E_CI = TANGENT_YOUNG_MODULUS(F_CK, 1)
E_CS = SECANT_YOUNG_MODULUS(F_CK, E_CI)
print("E_ci: ", "%.4f" % E_CI, "MPa", "\nE_cs: ", "%.4f" % E_CS, "MPa")

carga permanente linear:  10.7000 kN/m 
carga variável linear:  1.5000 kN/m
E_ci:  30672.4632 MPa 
E_cs:  26838.4053 MPa


In [3]:
STRUCTURE = {
  "TYPE_ELEMENT": 0,
  "TYPE_SOLUTION": 0,
  "N_NODES": 23,
  "N_MATERIALS": 1,
  "N_SECTIONS": 3,
  "N_ELEMENTS": 24,
  "N_DISPLACEMENTS": 3,
  "N_ELEMENTSLOADED": 0,
  "N_NODESLOADED": 1,
  "N_SPRINGS": 0,
  "COORDINATES": 
  np.array([[0.0, 0.0],
    [2.0, 0.0],
    [0.0, 2.0]]),
  "ELEMENTS": 
  np.array([[0, 1, 0, 0, 1, 1],
    [1, 2, 1, 1, 1, 1],
    [0, 2, 2, 2, 1, 1]]),
  "MATERIALS": 
  np.array([[2.050E8,0.3,78600,0.000012],
    [2.050E8,0.3,78600,0.000012],
    [2.050E8,0.3,78600,0.000012]]),
  "SECTIONS": 
  np.array([[0.0079,0.000005,0.000005,0.05,0.05],
    [0.0079,0.000005,0.000005,0.05,0.05],
    [0.0079,0.000005,0.000005,0.05,0.05]]),
  "PRESCRIBED DISPLACEMENTS": 
  np.array([[0,0,0],
    [0,1,0],
    [2,0,0]]),
  "ELEMENT LOADS": None,
  "NODAL LOADS": 
  np.array([[1,1,-Q]]),
  "SPRINGS": None}

In [4]:
RESULTS_G = MEF1D(DICTIONARY = TRUSS_01_G)

NameError: name 'TRUSS_01_G' is not defined

In [ ]:
# ANÁLISE ESTRUTURAL CARGA G
RESULTS_Q = MEF1D(DICTIONARY = TRUSS_01_Q)